<table style="font-size: 1em; padding: 0; margin: 0;">

<tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #182AEB; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #ffffff;">The Coastal Grain Size Portal (C-GRASP) dataset <br/><em>Will Speiser, Daniel Buscombe, Evan Goldstein</em></strong><br/><br/>
        <strong>> Compute Sample Distance from Shore </strong><br/>
    </p>                       
        
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">The purpose of this notebook</strong>  
    <br/><font color=grey> This notebook will output a dataframe containing a new field with an estimated of the distance between each sample from a chosen C-GRASP dataset and the coastline. As C-Grasp file sizes vary completion of this task will vary with internet connectivity and computer processing power.<font><br/>
    <br/><font color=grey> This notebook provides simple code that calculates the distance between each sample from a chosen CGRASP dataset and the NaturalEarth 1:50m physical coastline polyline.<font><br/>    
    <br/><font color=grey> To do so, a user can choose a dataset of choice. This dataset is then called to your system and then converted into a GeoDataFrame<font><br/>
    <br/><font color=grey> The notebook then uses Cartopy to call the NaturalEarth 1:10m coastline polygon<font><br/>
     <br/><font color=grey> This coastline is then converted to a GeoDataFrame object and is cropped to an extent of the Eastern United States<font><br/>  
    <br/><font color=grey> This coastline is then converted to a GeoDataFrame object and is cropped to an extent of the Eastern United States<font><br/> 
    <br/><font color=grey> Finally, using the GeoPandas distance function, the distance between nearest features in both GeoDataFrames (the chosen samples and the coastline polylines) is calculated and added to a new "Distance" field for each sample<font><br/>  
    </p>
    

In [ ]:
import pandas as pd
import geopandas as gpd
import shapefile
import json 
import ipywidgets
from cartopy.feature import NaturalEarthFeature
import cartopy.io.shapereader as shpreader
import numpy as np


#### Select a dataset

In [ ]:
#Dataset collection widget
zen=ipywidgets.Select(
    options=['Entire Dataset', 'Estimated Onshore Data', 'Verified Onshore Data', 'Verified Onshore Post 2012 Data'],
    value='Entire Dataset',
    # rows=10,
    description='Dataset:',
    disabled=False
)

display(zen)


#### Download the dataset

In [ ]:
url = 'https://zenodo.org/record/6099266/files/' 
if zen.value=='Entire Dataset':
    filename='dataset_10kmcoast.csv'
if zen.value=='Estimated Onshore Data':
    filename='Data_EstimatedOnshore.csv'
if zen.value=='Verified Onshore Data':
    filename='Data_VerifiedOnshore.csv'
if zen.value=='Verified Onshore Post 2012 Data':
    filename='Data_Post2012_VerifiedOnshore.csv'
print("Downloading {}".format(url+filename))   


The next cell will download the CGRASP dataset and read it in as a pandas dataframe with variable name `df`

In [ ]:
url=(url+filename)
print('Retrieving Data, Please Wait')
#retrieve data
df=pd.read_csv(url)
print('Sediment Data Retrieved!') 


Let's take a quick look at the top of the file

In [ ]:
df.head()

Now we'll convert that data frame to a GeoDataFrame, that way we can do spatial calculations on it.

In [ ]:
gdf=gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf=gdf.set_crs(epsg=4326)
print('Sediment Data Converted to a GeoDataFrame, Next cell retrieves Natural Earth Data')


Now let's call that NaturalEarth Coastline polyline:

In [ ]:
coast = shpreader.natural_earth(resolution='10m',category='physical', name='coastline')
coast=gpd.read_file(coast)
print('Natural Earth Data retrieved and converted to a GeoDataFrame')

This next cell creates a json to define bounds for the East Coast US (C-Grasp's area of study)

In [ ]:
#Define Bounds for the East Coast to Clip US Coastal Poly
eastcoast=json.loads(("""
     {"type": "FeatureCollection", "features": [{
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -97.294921875,
              24.766784522874453
            ],
            [
              -94.130859375,
              22.51255695405145
            ],
            [
              -86.30859375,
              21.94304553343818
            ],
            [
              -80.244140625,
              16.130262012034756
            ],
            [
              -65.0390625,
              15.792253570362446
            ],
            [
              -66.610107421875,
              44.66083904265621
            ],
            [
              -66.7694091796875,
              44.8500274926005
            ],
            [
              -66.8902587890625,
              44.779885502772736
            ],
            [
              -67.5,
              47.69497434186282
            ],
            [
              -69.60937499999999,
              47.754097979680026
            ],
            [
              -71.630859375,
              45.521743896993634
            ],
            [
              -101.25,
              30.221101852485987
            ],
            [
              -97.294921875,
              24.766784522874453
            ]
          ]
        ]
      }
    }]}
"""))


Here, we crop that NaturalEarth coastline with the East Coast Json

In [ ]:
eastcoast = gpd.GeoDataFrame.from_features(eastcoast) #convert east coast json to geoadataframe
eastcoast=eastcoast.set_crs(epsg=4326) 
eastcoast_line=gpd.clip(coast, eastcoast) #clip natural earth data to east coat
eastcoast = eastcoast_line.reset_index(drop=True)

In this next cell we will convert both all of the spatial data to EPSG 3857 crs so our distance measurements are in meters

In [ ]:
eastcoast=eastcoast.to_crs(epsg=3857)#project to 3857 to make calculation in meters rather than degrees
gdf=gdf.to_crs(epsg=3857)#project to 3857 to make calculation in meters rather than degrees
gdf = gdf.reset_index(drop=True)

This next cell runs the distance calculation

In [ ]:
gdf['Distance']=gdf.geometry.apply(lambda x: eastcoast.distance(x).min()) #calculate distance using geopandas and set it to df

Let's see how that worked:

In [ ]:
print(gdf['Distance'])

Now lets turn the dataframe back to a Pandas dataframe and drop the "geometry" column

In [ ]:
df=pd.DataFrame(gdf) #convert geodataframe to pandas data frame
df=df.drop(columns='geometry')

### Write to file

Finally, define a csv file name for the output dataframe

In [ ]:
output_csvfile='../data_plus_locations.csv'

write the data to that csv file

In [ ]:
df.to_csv(output_csvfile)